In [ ]:
def fix_eta(self,mass,sigmaE,fdm,idname,outdir):
        import numpy as np
        import os
        # os.system('rm -r ./data/*')
        fname_rhoDam = f'./results/results/{idname}.rho'
        fdata = np.loadtxt(fname_rhoDam,delimiter='\t')
        rho = fdata[:,1]
        mass_str = str(np.round(mass,3)).replace('.','_')
        if fdm == 0:
            dirname = 'Parameter_Scan_Scr'
        elif fdm == 2:
            dirname = 'Parameter_Scan_LM'

        write_dir = outdir + dirname + f'/mDM_{mass_str}_MeV_sigmaE_{sigmaE}_cm2/'
        if not os.path.isdir(write_dir):
            os.mkdir(write_dir)
        for isoangle in range(0,self.isorings):
            fname_DAMASCUS = f'./results/results/{idname}_histograms/eta.{isoangle}'
            
            fdata = np.loadtxt(fname_DAMASCUS,delimiter='\t')
            vmin = fdata[:,0] * self.s/self.km
            eta = fdata[:,1]*self.km/self.s
            eta_err = fdata[:,3]*self.km/self.s
            eta*=(rho[isoangle]/self.rhoX)
            # print(rho[isoangle]/self.rhoX,self.rhoX,rho[isoangle])
            eta_err*=(rho[isoangle]/self.rhoX)
            import csv
            with open(write_dir+f'DM_Eta_theta_{isoangle}.txt','w') as f:
                writer = csv.writer(f,delimiter='\t')
                writer.writerows(zip(vmin,eta,eta_err))
        # os.system(f"rm -r ./results/{idname}*")

In [ ]:
import os
from DaMaSCUS_helper import DaMaSCUS_helper
damascus = DaMaSCUS_helper()
import re
outdir = '../sensei_toy_limit/python/theory_tools/QEDark/halo_data/modulated/'
files = os.listdir('results')
for f in files:
    if 'histograms' not in f:
        continue
    if 'mX' not in f:
        continue
    mass = re.findall('mX.*si',f)[0][2:-3]
    mass = float(mass.replace('_','.'))
    sigmaP = float(re.findall('ma.*fdm',f)[0][2:-4])
    sigmaE = damascus.sigmaP_to_sigmaE(sigmaP,mass)

    sigmaE =float(format(sigmaE, '.2g'))

    fdm = re.findall('fdm.*.cfg',f)[0][3:-4]
    if fdm == 'none':
        continue
    else:
        fdm = int(fdm)
    idname = f[:-11]
    damascus.fix_eta(mass,sigmaE,fdm,idname,outdir,delete=False)
    # damascus.fix_eta(mass,sigmaE,fdm)
# fix_eta(mass,sigmaE,fdm,idname,outdir)

